In [65]:
import vk_api
import facebook
import requests
from keras.models import load_model
import pickle
import sklearn
import os
import tqdm
import numpy as np
from sklearn.preprocessing import normalize
import json
import pandas as pd
import pymorphy2
from nltk.tokenize import RegexpTokenizer
import re
import itertools

In [66]:
a = eval("""
{'Здравоохранение': 0.015024971670793637, 'Военное дело': 0.0062534099970621565, 'Безопасность': 0.01481512569773786, 'Стратегическое управление': 0.08645654089898015, 'Государственное управление': 0.05552524447055861, 'Управление персоналом': 0.045620514542325935, 'Исследования и разработки': 0.09367524237209889, 'Юриспруденция': 0.01611617073068368, 'Управление рисками': 0.02551727032358249, 'Сельское хозяйство': 0.011877282074956981, 'Социальное предпринимательство': 0.033491417299702016, 'Инновации и модернизация': 0.10693750786922399, 'Частный бизнес': 0.06140093171612037, 'Дошкольное образование/детский отдых': 0.012506819994124313, 'Строительство': 0.017501154152851806, 'Журналистика': 0.006001594829395224, 'Корпоративное управление': 0.05682628950350443, 'Образование': 0.10748310739916901, 'Искусство': 0.017417215763629496, 'Промышленность': 0.04839048138666219, 'Социальная защита': 0.009317161203676502, 'Политика': 0.02451000965291476, 'Финансы': 0.036513199311705206, 'Инвестиции': 0.050908633063331515, 'Реклама и маркетинг': 0.0399127040752088}""")

In [67]:
model_dist = {}
for col in known.columns[4:]:
    model_dist[norm_names[col]] = np.mean(known.loc[:, col])

In [69]:
corpora_old = pickle.load(open("corpora_old.p", "rb"))

In [76]:
corpora_old[6]

['друг',
 'форум армия проводить конкурс',
 'возможность выиграть три пригласительный билет международный военный технический форум армия',
 'участие розыгрыш',
 'быть подписать страница образование инновация рф',
 'подписаться страница форум армия',
 'сделать репост этот запись страница',
 'результат конкурс быть подвести август',
 'три победитель быть выбрать случайный образ помощь генератор случайный число',
 'весь удача',
 'событие розыгрыш конкурс форумармия кубинка форум',
 'исследование природа космический луч',
 'специалист институт ядерный физика имя скобельцын ниияф мгу имя метр ломоносов представить результат первое год работа орбита обсерватория космический луч нуклон ход эксперимент нуклон удаться измерить энергетический спектр различный химический элемент состав космический луч поток зарядить частица приходящий глубина вселенная время проведение космический эксперимент нуклон борт реализовать два метод измерение энергия космический частица позволить сравнивать результат д

In [68]:
model_dist

{'Безопасность': 0.29942477,
 'Военное дело': 0.14746161,
 'Государственное управление': 0.17855562,
 'Здравоохранение': 0.11960617,
 'Инвестиции': 0.097886823,
 'Инновации и модернизация': 0.072102837,
 'Искусство': 0.28367212,
 'Исследования и разработки': 0.19452339,
 'Корпоративное управление': 0.18265143,
 'Образование': 0.12748668,
 'Политика': 0.22378828,
 'Предпринимательство': 0.092833035,
 'Промышленность': 0.1561019,
 'Реклама и маркетинг': 0.21646775,
 'Сельское хозяйство': 0.26292121,
 'Социальная защита': 0.1898212,
 'Спорт': 0.1896285,
 'Стратегическое управление': 0.23626328,
 'Строительство': 0.25087744,
 'Финансы': 0.20739107,
 'Юриспруденция': 0.19676548}

In [54]:
sklearn.preprocessing.scale(list(model_dist.values()))

array([ 0.80898927,  0.63720682,  1.64596134,  0.65796093, -0.6504266 ,
       -0.23282708, -0.92511605, -0.6087226 ,  2.32348243, -0.72998129,
       -0.49487713, -0.17831297, -0.63775345, -1.14913023, -0.54427495,
       -0.37388981, -0.71070244,  0.72901701, -0.72374578,  2.07731984,
       -0.92017728])

In [12]:
class CorporaClass:
    def __init__(self):
        self.corpora = []
        self.vocab = set()

    tokenizer = RegexpTokenizer('\w+')
    morph = pymorphy2.MorphAnalyzer()
    ru_pattern = re.compile("[а-яА-Я]")

    @staticmethod
    def full_process(text, tokenizer=tokenizer, morph=morph, ru_pattern=ru_pattern):
        # Clear text from punctuation etc.'''
        tokens = tokenizer.tokenize(text)

        # Turn tokens into normal form excluding non-nouns or verbs
        processed = []
        for token in tokens:
            morphed = morph.parse(token)[0].normal_form
            nf_tag = str(morph.parse(morphed)[0].tag.POS)
            if nf_tag in ("NOUN", "ADJF", "INFN", "NUMR") and len(token) < 16:
                if len(morphed) == len(re.findall(ru_pattern, morphed)):
                    processed.append(morphed)

        result = " ".join(processed)
        return result

    def add_to_corpora(self, file_object):
        try:
            doc = []
            for line in file_object:
                try:
                    processed = self.full_process(line)
                except Exception as e:
                    print(e)
                    processed = ""
                if len(processed):
                    doc.append(processed)
            stoplist = pd.Series(list(itertools.chain(*(a.split() for a in doc)))).value_counts().index[:5]
            print(f"Excluded: {list(stoplist)}")
            accepted_lines = []
            for line in doc:
                accepted_words = []
                for word in line.split():
                    if word not in stoplist:
                        accepted_words.append(word)
                        self.vocab.add(word)
                accepted_lines.append(" ".join(accepted_words))
            self.corpora.append(accepted_lines)
        except Exception:
            pass
        self.vocab = self.vocab - {""}

In [13]:
VK_TOKEN = "2be03a7932fccb910597649a6a9b2609f1931d809c6f4278c0ed1d50f6774ecd792611d900206b10dcdb1"
FB_TOKEN = "EAABtRlqbz5YBAFuHv18RyKN8ZCJiMZARqzPsfCADT4eZApjulZA9hxcBt0vaILFY9jRfgO5KX6ZBsM27xUn3kMmFQQXcGpCYFLjEnVZB3ObRN4dZAXaiOOK0OyJogthersprvYNrzkncCKwoDS7116DRIwm8a9Dead7Ef0gVpxa8m5vxSTrcCRt9IQbGMs8EE8VqM9kZANo6bAZDZD"

In [19]:
def get_posts_fb(user='BillGates'):

    # You'll need an access token here to do anything.  You can get a temporary one
    # here: https://developers.facebook.com/tools/explorer/
    path = f"corpora_from_fb_users/{user}.txt"
    if not os.path.exists(path):
        access_token = FB_TOKEN

        graph = facebook.GraphAPI(access_token)
        profile = graph.get_object(user)
        posts = graph.get_connections(profile['id'], 'posts')
        
        with open(path, 'w') as f:
            texts = []
            while True:
                try:
                    # Perform some action on each post in the collection we receive from
                    # Facebook.
                    for post in posts['data']:
                        msg = post.get('message', '')
                        if msg:
                            processed_text = full_process(msg)
                            texts.append(processed_text)
                            _ = f.write(f"{processed_text}\n")
                    # Attempt to make a request to the next page of data, if it exists.
                    posts = requests.get(posts['paging']['next']).json()
                except KeyError:
                    # When there are no more pages (['paging']['next']), break from the
                    # loop and end the script.
                    break
    with open(path) as f:
        return list(f)
        
def get_posts_fb_temp(user='BillGates'):

    # You'll need an access token here to do anything.  You can get a temporary one
    # here: https://developers.facebook.com/tools/explorer/
    path = f"corpora_from_fb_users/{user}.txt"
    if not os.path.exists(path):
        with open(path, 'w') as f:
            texts = []
            for post in t[user]:
                if post:
#                     processed_text = full_process(post)
                    processed_text = post
                    texts.append(processed_text)
                    _ = f.write(f"{processed_text}\n")
    with open(path) as f:
        return list(f)

In [20]:
import json
t = json.load(open("fb_dump.json"))

In [21]:
def getallwall(kwargs, n=None):
    '''Get all texts from wall generator'''
    vk_session = vk_api.VkApi(token=VK_TOKEN)
    tools = vk_api.VkTools(vk_session)
    if n:
        try:
            wall_posts = tools.get_all_iter("wall.get", 75, values=kwargs, limit=n)
            for i, post in enumerate(wall_posts, 1):
                if i > n:
                    break
                yield post['text']
        except:
            wall_posts = tools.get_all_iter("wall.get", 15, values=kwargs, limit=n)
            for i, post in enumerate(wall_posts, 1):
                if i > n:
                    break
                yield post['text']
    else:
        try:
            wall_posts = tools.get_all_iter("wall.get", 75, values=kwargs)
            for post in wall_posts:
                yield post['text']
        except:
            wall_posts = tools.get_all_iter("wall.get", 15, values=kwargs)
            for post in wall_posts:
                yield post['text']

def get_posts_vk(user, n_wall=None):
    path = f"corpora_from_vk_users/{user}.txt"
    if not os.path.exists(path):
        wall = getallwall({"owner_id": user}, n_wall)
        texts = []
        with open(path, "w") as f:
            for post in wall:
                if post:
                    _ = f.write(f"{processed_text}\n")
    with open(path) as f:
        return list(f)
        
def process_public_vk(public_id, n_wall=None):
    texts = []
    path = f"corpora_from_vk_publics/{public_id}.txt"
    if not os.path.exists(path):
        wall = getallwall({"owner_id": -public_id}, n_wall)
        with open(path, "w") as f:
            for post in wall:
                if post:
                    _ = f.write(f"{post}\n")
    with open(path) as f:
        return list(f)
        
def get_publics(user_id, num_publics):
    vk_session = vk_api.VkApi(token=VK_TOKEN)
    vk = vk_session.get_api()
    groups = vk.groups.get(user_id=user_id, extended=1, fields='members_count', count=25)['items']
    return [g['id'] for g in groups if g.get('members_count', 1000000) < 1000000][:num_publics]

In [57]:
categories = json.load(open("categories.json"))
classifier = load_model("vk_texts_classifier.h5")
vectorizer = pickle.load(open("vectorizer.p", "rb"))

def get_result(user_vk, user_fb, categories=categories, classifier=classifier, vectorizer=vectorizer):
    texts = []
    if user_vk:
        texts.extend(get_posts_vk(user_vk))
        public_ids = get_publics(user_vk, 6)
        for public_id in public_ids:
            texts.extend(process_public_vk(public_id, 2000))
    if user_fb:
#         texts.extend(get_posts_fb(user_fb))    
        texts.extend(get_posts_fb_temp(user_fb))
    processed_texts = []
    for text in texts:
        processed_text = full_process(text)
        if len(processed_text):
            processed_texts.append(processed_text)
    
    stoplist = pd.Series(list(itertools.chain(*(a.split() for a in processed_texts)))).value_counts().index[:50]
    accepted_lines = []
    for line in processed_texts:
        accepted_words = []
        for word in line.split():
            if word not in stoplist:
                accepted_words.append(word)
        accepted_lines.append(" ".join(accepted_words))
    
    verdict = normalize(np.sum(classifier.predict(vectorizer.transform(accepted_lines).toarray()), axis=0).reshape(1, -1))[0]
    return list(zip(categories, verdict))

In [58]:
tqdm.tqdm.pandas()

In [59]:
import warnings
warnings.filterwarnings("ignore")

In [60]:
# social = pd.read_excel("social.xlsx", names=['id', 'name', 'fb', 'ok', 'vk', 'email', 'twitter'])
# social.set_index('id', inplace=True)
# social = social[social.vk.notnull()]
# social.vk = social.vk.apply(lambda x: str(x).split("/")[-1])
# df = social[social.vk.notnull()]
# df = df[['name', 'vk']]

# def get_id(screen_name):
#     try:
#         item = vk.users.get(user_ids=screen_name)
#         if 'deactivated' not in item:
#             return item[0]['id']
#     except Exception as e:
#         print(e.args)
# # Нужно замапить screen_name с их vk_id, поэтому придется обрабатывать по одному

# df['vk_id'] = df.vk.progress_apply(get_id)
# df.dropna(inplace=True)
# df.vk_id = df.vk_id.astype('int')
# df.to_msgpack("df.msg")
df = pd.read_msgpack("df.msg")

In [61]:
known = pd.read_csv("known_users.csv")
known = known.merge(df[['vk_id']], left_on='Leader-ID', right_index=True, how='left')

In [62]:
offset = 0
for index, row in tqdm.tqdm(known.iloc[offset:, :].iterrows(), total=len(known) - offset):
    user_vk = row['vk_id']
    if np.isnan(user_vk):
        user_vk = None
    user_fb = row['FB']
    try:
        verdict = get_result(user_vk, user_fb)
        for cat, value in verdict:
            known.loc[index, cat] = value
    except Exception as e:
        print(e.args)
        for cat in categories:
            known.loc[index, cat] = 0

 34%|███▍      | 15/44 [00:35<00:54,  1.88s/it]

('Found array with 0 sample(s) (shape=(0, 10116)) while a minimum of 1 is required by the normalize function.',)


100%|██████████| 44/44 [02:31<00:00, 13.33s/it]


In [63]:
norm_names = dict(zip(categories, 
         ["Искусство", "Политика", "Финансы", "Стратегическое управление", "Юриспруденция", "Исследования и разработки",
          "Промышленность", "Образование", "Социальное предпринимательство", "Здравоохранение", "Сельское хозяйство", 
          "Государственное управление", "Реклама и маркетинг", "Инновации и модернизация", "Безопасность", 
          "Военное дело", "Корпоративное управление", "Социальная защита", "Строительство", "Предпринимательство",
          "Спорт", "Инвестиции"]))

In [64]:
norm_dict = {"values": []}
for name in known.ФИ:
    results = []
    for col in known[known.ФИ == name].iloc[:, 4:].columns:
        results.append({"name": norm_names[col], "value": float(known.loc[known.ФИ == name, col].values[0])})
    norm_dict['values'].append({"name": name, "results": results})

In [35]:
known.loc[known.FB == 'peskov', 'EDUCATION']

41    0.106072
Name: EDUCATION, dtype: float32

In [31]:
json.dump(norm_dict, open("temporary_result.json", "w"))

In [239]:
corpora = pickle.load(open("corpora.p", "rb"))

In [242]:
vectorizer = pickle.load(open("vectorizer.p", "rb"))
vectorizer_old = pickle.load(open("vectorizer_old.p", "rb"))

In [254]:
len(vectorizer.vocabulary_)
len(vectorizer_old.vocabulary_)

87158

119411

In [256]:
competencies = pd.read_csv("competences.csv")

In [257]:
competencies

,Id,Интересы,Организация работы,Обмен информацией и коммуникации,Организация собственной работы и поддержание необходимого уровня работоспособности,Обучение и развитие подчиненных,Подготовка и реализация управленческих решений,Выполнение административных функций,Обеспечение должного уровня мотивации подчиненных,Оценка и контроль работы подчиненных,...,Схематизация,Открытое мышление,Системное видение,Командная работа,Построение отношений,Нацеленность на достижения,Лидерские качества,Модерация рабочих групп,Проведение игропрактических мероприятий,Разработка игропрактических форматов
0,81339,NaN,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11890,"Образование, Исследования и разработки, Иннова...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,103294,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,79271,NaN,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,111839,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,59145,NaN,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,75325,NaN,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
7,22262,"Юриспруденция, Государственное управление, Час...",3,7,5,0,0,0,0,0,...,0,0,0,0,5,3,0,0,0,0
8,70276,NaN,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,87832,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0


In [236]:
known

,ФИ,Leader-ID,FB,vk_id,ART,POLITICS,FINANCES,STRATEG,LAW,ELABORATION,...,GOVERNMENT,SMM,INNOVATIONS,SAFETY,MILITARY,CORPORATIVE,BUILDING,ENTREPRENEURSHIP,SPORT,INVESTITIONS
0,Александрина Клюс,91500,alya.blesk,NaN,0.351330,0.090132,0.301443,0.177843,0.040636,0.208065,...,0.065257,0.589567,0.243945,0.062870,0.229396,0.077893,0.141131,0.101688,0.263744,0.102868
1,Ангелина Зонова,82050,zonovaangelina,NaN,0.090223,0.064310,0.053899,0.073481,0.072817,0.102575,...,0.096668,0.011020,0.075028,0.012868,0.009560,0.083595,0.572881,0.580815,0.053287,0.225407
2,Владислав Широков,102575,100018653177414,NaN,0.084959,0.157256,0.000046,0.379977,0.003912,0.895085,...,0.003133,0.000199,0.021707,0.003932,0.076989,0.084202,0.011795,0.003084,0.021829,0.018277
3,Галина Жукова,44515,1088734051,NaN,0.835085,0.108337,0.049757,0.158968,0.028548,0.176231,...,0.038733,0.074972,0.206898,0.082150,0.182496,0.058435,0.124629,0.096256,0.132520,0.074480
4,Жанна Кадылева,584,kadilevazm,NaN,0.496938,0.109778,0.318557,0.184312,0.041818,0.169685,...,0.034906,0.304515,0.344391,0.076757,0.349330,0.070670,0.080990,0.046369,0.103039,0.066043
5,Ирина Горькова,NaN,100013685044932,NaN,0.421183,0.437242,0.019236,0.206018,0.111381,0.318001,...,0.007143,0.044846,0.134943,0.062215,0.254062,0.464772,0.162257,0.026987,0.100580,0.112958
6,Ирина Григоренко,39,i.g.grigorenko,NaN,0.017518,0.026473,0.855746,0.071599,0.013906,0.129585,...,0.010374,0.097426,0.017029,0.024737,0.393153,0.027238,0.024095,0.116779,0.005602,0.177334
7,Наталья Кульбятская,12271,nvkulbyatskaya,NaN,0.661575,0.101562,0.090342,0.162755,0.152673,0.156673,...,0.028038,0.363517,0.285062,0.189863,0.205382,0.102061,0.136221,0.024793,0.178809,0.052717
8,Нелли Бадалян,16662,100017461808545,NaN,0.974204,0.077492,0.000264,0.098443,0.003198,0.080830,...,0.030636,0.000478,0.034562,0.005471,0.094735,0.016989,0.081154,0.001411,0.093570,0.004907
9,Юлия Ханьжина,1419,1496282246,NaN,0.613292,0.159208,0.310500,0.193205,0.043620,0.213379,...,0.112200,0.087711,0.273597,0.117873,0.233450,0.080412,0.123211,0.089749,0.257510,0.063560


In [237]:
known.to_csv("known2.csv")

In [ ]:
norm_names

In [69]:
offset = 230
for index, row in tqdm.tqdm(df.iloc[offset:, :].iterrows(), total=len(df) - offset):
    user_id = row['vk_id']
    try:
        public_ids = get_publics(user_id, 5)
        verdict = process_publics(public_ids, 300)
        for cat, value in verdict:
            df.loc[index, cat] = value
    except Exception as e:
        print(e.args)
        for cat in categories:
            df.loc[index, cat] = 0


  2%|▏         | 17/708 [14:00<5:32:20, 28.86s/it] 

('Found array with 0 sample(s) (shape=(0, 119411)) while a minimum of 1 is required by the normalize function.',)


  3%|▎         | 19/708 [15:03<5:15:04, 27.44s/it]

('Found array with 0 sample(s) (shape=(0, 119411)) while a minimum of 1 is required by the normalize function.',)


  3%|▎         | 21/708 [16:09<5:12:37, 27.30s/it]

('Found array with 0 sample(s) (shape=(0, 119411)) while a minimum of 1 is required by the normalize function.',)


  7%|▋         | 48/708 [33:48<5:42:05, 31.10s/it] 

('Found array with 0 sample(s) (shape=(0, 119411)) while a minimum of 1 is required by the normalize function.',)


  7%|▋         | 52/708 [36:06<6:44:25, 36.99s/it]

KeyboardInterrupt: 

In [70]:
df.to_msgpack("df.msg")

In [71]:
df.to_csv("df.csv")